In [58]:
import mne
import pandas as pd
from scipy.fft import fft, fftfreq
import scipy

In [51]:
def create_raw_object(file_path):
    # Creating the raw edf object using mne library  
    return mne.io.read_raw_edf(file_path)

In [52]:
def visualize_data(raw_object, duration_sec, start_time, eeg_color):
    # Plotting the EEG data   
    raw_object.plot(duration=duration_sec, start=start_time, color=dict(mag='black', grad='b', eeg=eeg_color, eog='k', ecg='m',
     emg='k', ref_meg='steelblue', misc='k', stim='k',
     resp='k', chpi='k'), bad_color="maroon")

In [53]:
def xlsx_to_df(file_path):
    train_seizures_timing = pd.read_excel(
        file_path,
        sheet_name='train',
        header=0,
        names=['Patient', 'Session', 'File', 'EEG Type', 'Normal/Abnormal', 'Start', 'Stop', 'Seizure Type'],
        usecols='C:F, I, M:O'
    )
    
    dev_seizures_timing = pd.read_excel(
        file_path,
        sheet_name='dev',
        header=0,
        names=['Patient', 'Session', 'File', 'EEG Type', 'Normal/Abnormal', 'Start', 'Stop', 'Seizure Type'],
        usecols='C:F, I, M:O'
    )
    
    
    return train_seizures_timing, dev_seizures_timing

In [54]:
train_seizures_timing, dev_seizures_timing = xlsx_to_df('_DOCS/seizures_v36r.xlsx')

In [46]:
train_seizures_timing.head()

,Patient,Session,File,EEG Type,Normal/Abnormal,Start,Stop,Seizure Type
0,NaN,NaN,NaN,NaN,NaN,Start,Stop,NaN
1,1402.0,s003,t001,Outpatient,Normal,NaN,NaN,NaN
2,1479.0,s001,t001,ICU,Abnormal,950.004,1047.02,FNSZ
3,1413.0,s001,t000,EMU,Abnormal,315.34,319.024,ABSZ
4,NaN,NaN,NaN,NaN,NaN,327.668,330.032,ABSZ


In [55]:
rawEDF = create_raw_object("edf_train_01_tcp_ar_000_00000077_s003_2010_01_21_00000077_s003_t000.edf")

Extracting EDF parameters from /home/sophia/eeg_epilepsy/edf_train_01_tcp_ar_000_00000077_s003_2010_01_21_00000077_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [56]:
rawEDF.info

<Info | 7 non-empty values
 bads: []
 ch_names: EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-REF, EEG C3-REF, ...
 chs: 36 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: 2010-01-21 13:58:36 UTC
 nchan: 36
 projs: []
 sfreq: 250.0 Hz
>

In [72]:
start, stop = rawEDF.time_as_index([100, 101])
print(start, stop)
data, times = rawEDF[0:33:1, start:stop]

25000 25250
